## 분류

### KNN 알고리즘

In [1]:
'''
# KNN (K-Nearest-Keighbors) : K 개의 가까운 이웃 
# - 새로운 관측값 주어지면 기존 데이터 중에서 가장 속성이 비슷한 K개의 이웃을 먼저 찾음.
    그 가까운 이웃들이 갖고 있느 목표값과 같은 값으로 분류하여 예측하는 알고리즘
    
# K값에 다라 예측의 정확도가 달라지므로, 적절한 K값을 찾는 것이 매우 중요하다
'''

'\n# KNN (K-Nearest-Keighbors) : K 개의 가까운 이웃 \n# - 새로운 관측값 주어지면 기존 데이터 중에서 가장 속성이 비슷한 K개의 이웃을 먼저 찾음.\n    그 가까운 이웃들이 갖고 있느 목표값과 같은 값으로 분류하여 예측하는 알고리즘\n    \n# K값에 다라 예측의 정확도가 달라지므로, 적절한 K값을 찾는 것이 매우 중요하다\n'

In [2]:
import pandas as pd
import seaborn as sns

In [4]:
# 데이터 로딩
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [5]:
# 데이터 탐색
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [6]:
# 데이터 전처리
# NaN이 너무 많은 deck열, 중복되는 embark_town 열 삭제
rdf = df.drop(['deck', 'embark_town'], axis=1)
rdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   survived    891 non-null    int64   
 1   pclass      891 non-null    int64   
 2   sex         891 non-null    object  
 3   age         714 non-null    float64 
 4   sibsp       891 non-null    int64   
 5   parch       891 non-null    int64   
 6   fare        891 non-null    float64 
 7   embarked    889 non-null    object  
 8   class       891 non-null    category
 9   who         891 non-null    object  
 10  adult_male  891 non-null    bool    
 11  alive       891 non-null    object  
 12  alone       891 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 72.5+ KB


In [7]:
rdf.columns.values

array(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'alive', 'alone'],
      dtype=object)

In [8]:
# age열에서 NaN인 행을 삭제(복습 할 때는 평균값으로 치환해서 수행해보세요)
rdf = rdf.dropna(subset=['age'],how='any', axis=0)
len(rdf)

714

In [10]:
# embarked 열의 NaN 값을 embarked에서 가장 많이 나온 도시값으로 치환
freq = rdf['embarked'].value_counts(dropna=True).idxmax()
freq

'S'

In [11]:
rdf.describe(include='all')

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
count,714.000000,714.000000,714,714.000000,714.000000,714.000000,714.000000,712,714,714,714,714,714
unique,NaN,NaN,2,NaN,NaN,NaN,NaN,3,3,3,2,2,2
top,NaN,NaN,male,NaN,NaN,NaN,NaN,S,Third,man,True,no,True
freq,NaN,NaN,453,NaN,NaN,NaN,NaN,554,355,413,413,424,404
mean,0.406162,2.236695,NaN,29.699118,0.512605,0.431373,34.694514,NaN,NaN,NaN,NaN,NaN,NaN
std,0.491460,0.838250,NaN,14.526497,0.929783,0.853289,52.918930,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,1.000000,NaN,0.420000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,1.000000,NaN,20.125000,0.000000,0.000000,8.050000,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,2.000000,NaN,28.000000,0.000000,0.000000,15.741700,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.000000,3.000000,NaN,38.000000,1.000000,1.000000,33.375000,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# 치환
rdf['embarked'].fillna(freq, inplace=True)

In [13]:
rdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   survived    714 non-null    int64   
 1   pclass      714 non-null    int64   
 2   sex         714 non-null    object  
 3   age         714 non-null    float64 
 4   sibsp       714 non-null    int64   
 5   parch       714 non-null    int64   
 6   fare        714 non-null    float64 
 7   embarked    714 non-null    object  
 8   class       714 non-null    category
 9   who         714 non-null    object  
 10  adult_male  714 non-null    bool    
 11  alive       714 non-null    object  
 12  alone       714 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 63.6+ KB


In [15]:
# 분석에 사용할 속성(설명변수) 선택

ndf = rdf[['survived','pclass','sex','age','sibsp','parch','embarked']]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [17]:
# one-hot encoding
onehot_sex = pd.get_dummies(ndf['sex'])
ndf=pd.concat([ndf, onehot_sex], axis=1)
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male
0,0,3,male,22.0,1,0,S,0,1
1,1,1,female,38.0,1,0,C,1,0
2,1,3,female,26.0,0,0,S,1,0
3,1,1,female,35.0,1,0,S,1,0
4,0,3,male,35.0,0,0,S,0,1


In [20]:
onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='embarked')
ndf = pd.concat([ndf,onehot_embarked], axis=1)
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male,embarked_C,embarked_Q,embarked_S
0,0,3,male,22.0,1,0,S,0,1,0,0,1
1,1,1,female,38.0,1,0,C,1,0,1,0,0
2,1,3,female,26.0,0,0,S,1,0,0,0,1
3,1,1,female,35.0,1,0,S,1,0,0,0,1
4,0,3,male,35.0,0,0,S,0,1,0,0,1


In [22]:
# 이제 머신러닝 모델이 읽지 못하는 열은 삭제처리
ndf.drop(['sex','embarked'], axis=1, inplace=True)
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,embarked_C,embarked_Q,embarked_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


In [23]:
# 독립변수(설명변수) X / 종속변수(목적변수) y

X = ndf[['pclass','age','sibsp','parch','female','male','embarked_C','embarked_Q','embarked_S']]
y = ndf['survived']



In [24]:
# 설명변수를 정규화
from sklearn.preprocessing import StandardScaler

X = StandardScaler().fit(X).transform(X)
X



array([[ 0.91123237, -0.53037664,  0.52457013, ..., -0.47180795,
        -0.20203051,  0.53307848],
       [-1.47636364,  0.57183099,  0.52457013, ...,  2.11950647,
        -0.20203051, -1.87589641],
       [ 0.91123237, -0.25482473, -0.55170307, ..., -0.47180795,
        -0.20203051,  0.53307848],
       ...,
       [-1.47636364, -0.73704057, -0.55170307, ..., -0.47180795,
        -0.20203051,  0.53307848],
       [-1.47636364, -0.25482473, -0.55170307, ...,  2.11950647,
        -0.20203051, -1.87589641],
       [ 0.91123237,  0.15850313, -0.55170307, ..., -0.47180795,
         4.94974747, -1.87589641]])

In [25]:
# 훈련용 데이터셋 / 테스트용 데이터셋 구분
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

print('train data shape : ', X_train.shape)
print('test data shape : ', X_test.shape)


train data shape :  (571, 9)
test data shape :  (143, 9)


In [26]:
# KNN 분류 모형으로 학습/예측/평가
from sklearn.neighbors import KNeighborsClassifier

# K 이웃을 5으로 임의로 설정
knn = KNeighborsClassifier(n_neighbors=5)

# train data 학습
knn.fit(X_train, y_train)

# test data로 클래스 예측(분류)
y_pred = knn.predict(X_test)
y_pred[:10]

array([0, 0, 1, 0, 0, 1, 1, 0, 0, 0], dtype=int64)

In [27]:
y_test[:10] # 간단히 예측 결과를 레이블(정답)을 통해 확인

728    0
555    0
426    1
278    0
617    0
751    1
576    1
679    1
567    0
117    0
Name: survived, dtype: int64

In [28]:
# 모형 성능 평가 - 오차행렬 체크
from sklearn.metrics import confusion_matrix


knn_martix = confusion_matrix(y_test, y_pred)
knn_martix

array([[75, 10],
       [18, 40]], dtype=int64)

In [29]:
# 평가 지표를 계산
from sklearn.metrics import classification_report
knn_report = classification_report(y_test, y_pred)
print(knn_report)

              precision    recall  f1-score   support

           0       0.81      0.88      0.84        85
           1       0.80      0.69      0.74        58

    accuracy                           0.80       143
   macro avg       0.80      0.79      0.79       143
weighted avg       0.80      0.80      0.80       143



## SVM 알고리즘

In [ ]:
'''
# SVM(Support Vector Machine)
 : 설명변수의 열 벡터는 각각의 고유의 축을 갖는 벡터 공간을 만들 수 있고, 분석 대상이 되는 개별 관측값을 모든 속성(열 백터)에
   관한 값을 해당 축의 좌표로 표시하여 벡터 공간에서의 위치로 나타내는 방법 
 : 만약 속성이 2개 존재하는 데이터셋이라면 2차원 평면공간, 3개라면 3차원 공간에,4개 이상이라면 고차원 벡터 공간 좌표를 사용
   할 수 있다.
   
# 학습방법 : 벡터 공간에 위치한 훈련 데이터의 좌표와 각 데이터가 어떤 분류값을 가져야 하는지 레이블을 입력 받아서 학습하는 형태   
'''

In [31]:
import pandas as pd
import seaborn as sns

In [32]:
# 데이터 로딩
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [33]:
# 데이터 탐색
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [34]:
# 데이터 전처리
# NaN이 너무 많은 deck열, 중복되는 embark_town 열 삭제
rdf = df.drop(['deck', 'embark_town'], axis=1)
rdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   survived    891 non-null    int64   
 1   pclass      891 non-null    int64   
 2   sex         891 non-null    object  
 3   age         714 non-null    float64 
 4   sibsp       891 non-null    int64   
 5   parch       891 non-null    int64   
 6   fare        891 non-null    float64 
 7   embarked    889 non-null    object  
 8   class       891 non-null    category
 9   who         891 non-null    object  
 10  adult_male  891 non-null    bool    
 11  alive       891 non-null    object  
 12  alone       891 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 72.5+ KB


In [35]:
# age열에서 NaN인 행을 삭제(복습 할 때는 평균값으로 치환해서 수행해보세요)
rdf = rdf.dropna(subset=['age'],how='any', axis=0)

# embarked 열의 NaN 값을 embarked에서 가장 많이 나온 도시값으로 치환
freq = rdf['embarked'].value_counts(dropna=True).idxmax()



714

In [38]:
# 치환
rdf['embarked'].fillna(freq, inplace=True)

In [39]:
# 분석에 사용할 속성(설명변수) 선택

ndf = rdf[['survived','pclass','sex','age','sibsp','parch','embarked']]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [40]:
# one-hot encoding
onehot_sex = pd.get_dummies(ndf['sex'])
ndf=pd.concat([ndf, onehot_sex], axis=1)


onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='embarked')
ndf = pd.concat([ndf,onehot_embarked], axis=1)
ndf.head()

# 이제 머신러닝 모델이 읽지 못하는 열은 삭제처리
ndf.drop(['sex','embarked'], axis=1, inplace=True)
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,embarked_C,embarked_Q,embarked_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


In [41]:
# 독립변수(설명변수) X / 종속변수(목적변수) y

X = ndf[['pclass','age','sibsp','parch','female','male','embarked_C','embarked_Q','embarked_S']]
y = ndf['survived']


# 설명변수를 정규화
from sklearn.preprocessing import StandardScaler

X = StandardScaler().fit(X).transform(X)
X.shape



(714, 9)

In [42]:
# 훈련용 데이터셋 / 테스트용 데이터셋 구분
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

print('train data shape : ', X_train.shape)
print('test data shape : ', X_test.shape)


train data shape :  (571, 9)
test data shape :  (143, 9)


In [44]:
# SVM 분류 모형 학습/예측/평가

from sklearn.svm import SVC

# kernel : 데이터를 벡터 공간으로 매핑해주는 함수 optrion
# fbf(radial basics function) | Linear | PoLynimal | Sigmoid...
svm_model = SVC(kernel='rbf')

svm_model.fit(X_train, y_train)

SVC()

In [45]:
y_pred = svm_model.predict(X_test)
y_pred[:10]
print('-----------------------')
y_test.values[:10]

-----------------------


array([0, 0, 1, 0, 0, 1, 1, 1, 0, 0], dtype=int64)

In [46]:
# 모델 평가/검증
from sklearn.metrics import confusion_matrix

svm_matrix = confusion_matrix(y_test,y_pred)
svm_matrix

array([[80,  5],
       [22, 36]], dtype=int64)

In [48]:
# 평가지표
from sklearn.metrics import classification_report
svm_report = classification_report(y_test, y_pred)
print(svm_report)

              precision    recall  f1-score   support

           0       0.78      0.94      0.86        85
           1       0.88      0.62      0.73        58

    accuracy                           0.81       143
   macro avg       0.83      0.78      0.79       143
weighted avg       0.82      0.81      0.80       143



### Decision Tree (결정트리)

In [49]:
import pandas as pd
import numpy as np

In [51]:
# 데이터 로딩
uci_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data'

df = pd.read_csv(uci_data, header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [ ]:
'''
# dataset decribe
샘플 id | 암세포 조직의 크기와 모양 등 종양 특성을 나태내는 열
'''

In [52]:
# 데이터 탐색
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       699 non-null    int64 
 1   1       699 non-null    int64 
 2   2       699 non-null    int64 
 3   3       699 non-null    int64 
 4   4       699 non-null    int64 
 5   5       699 non-null    int64 
 6   6       699 non-null    object
 7   7       699 non-null    int64 
 8   8       699 non-null    int64 
 9   9       699 non-null    int64 
 10  10      699 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.2+ KB


In [58]:
# 컬럼 이름 지정
df.columns = ['id','clump','cell_size','cell_shape','adhesion',' epithlial','bare_nucleoli','chromatin','normal_nucleoli','mitoses','class']
df.head()

,id,clump,cell_size,cell_shape,adhesion,epithlial,bare_nucleoli,chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [59]:
# 요약정보
df.describe()

,id,clump,cell_size,cell_shape,adhesion,epithlial,chromatin,normal_nucleoli,mitoses,class
count,6.990000e+02,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,1.071704e+06,4.417740,3.134478,3.207439,2.806867,3.216023,3.437768,2.866953,1.589413,2.689557
std,6.170957e+05,2.815741,3.051459,2.971913,2.855379,2.214300,2.438364,3.053634,1.715078,0.951273
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.706885e+05,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171710e+06,4.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238298e+06,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [60]:
# bare_nucleoli 열의 특이한 데이터를 확인
df['bare_nucleoli'].unique()

array(['1', '10', '2', '4', '3', '9', '7', '?', '5', '8', '6'],
      dtype=object)

In [63]:
# 데이터 전처리
# ? -> NaN -> NaN 삭제 -> object to int
df['bare_nucleoli'].replace('?', np.nan, inplace=True)
df.dropna(subset=['bare_nucleoli'], axis=0, inplace=True)
df['bare_nucleoli'] = df['bare_nucleoli'].astype('int')


In [64]:
df.describe()

,id,clump,cell_size,cell_shape,adhesion,epithlial,bare_nucleoli,chromatin,normal_nucleoli,mitoses,class
count,6.830000e+02,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000
mean,1.076720e+06,4.442167,3.150805,3.215227,2.830161,3.234261,3.544656,3.445095,2.869693,1.603221,2.699854
std,6.206440e+05,2.820761,3.065145,2.988581,2.864562,2.223085,3.643857,2.449697,3.052666,1.732674,0.954592
min,6.337500e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.776170e+05,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171795e+06,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238705e+06,6.000000,5.000000,5.000000,4.000000,4.000000,6.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 683 entries, 0 to 698
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   id               683 non-null    int64
 1   clump            683 non-null    int64
 2   cell_size        683 non-null    int64
 3   cell_shape       683 non-null    int64
 4   adhesion         683 non-null    int64
 5    epithlial       683 non-null    int64
 6   bare_nucleoli    683 non-null    int32
 7   chromatin        683 non-null    int64
 8   normal_nucleoli  683 non-null    int64
 9   mitoses          683 non-null    int64
 10  class            683 non-null    int64
dtypes: int32(1), int64(10)
memory usage: 61.4 KB


In [66]:
# 설명 변수 X / 목적 변수 y 구분

X = df[['clump','cell_size','cell_shape','adhesion',' epithlial','bare_nucleoli','chromatin','normal_nucleoli','mitoses']]

y = df['class']

In [70]:
# 정규화
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit(X).transform(X)

# 훈련 세트 / 데이터 세트 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

print('train data shape : ', X_train.shape)
print('test data shape : ', X_test.shape)


train data shape :  (512, 9)
test data shape :  (171, 9)


In [ ]:
'''
# Decision Tree 분류 모형
# 구조 : 컴퓨터 알고리즘에서 사용하는 트리(tree) 구조를 사용
       : 각 분기점(node)에는 분석대상의 속성(설명 변수)들이 위치하고, 해당 속성이 갖는 값을 이용하영 새로운 가지(branch)를 만드는 방법
       : 각 분기점에서 최적의 속성을 선택할 때는 해당 속성을 기준으로 분류한 값들이 구분되는 정도를 측정
       => 다른 종류의 값들이 섞여 있는 정도를 나타낼 때는 entropy를 주로 활용
       => 이 entropy가 낮을 수록 분류가 잘 되는 형태
       =>entropy가 일정 수준 이하로 낮아질 때 까지 과정을 반복 

'''

In [71]:
from sklearn.tree import DecisionTreeClassifier

# max_depth : 트레 레벨
# 레벨이 많아지면 정확해지지만 너무 많으면 과대적합 위험성 있음.

tree_model = DecisionTreeClassifier(criterion='entropy')

tree_model.fit(X_train, y_train)

y_pred = tree_model.predict(X_test)

print(y_pred[:10])
print('-----------------------------------------')
print(y_test.values[:10])

[4 4 4 4 4 4 2 2 4 4]
-----------------------------------------
[4 4 4 4 4 4 2 2 4 4]


In [72]:
# 평가
from sklearn import metrics

tree_matrix = metrics.confusion_matrix(y_test, y_pred)
tree_matrix

array([[106,   3],
       [  3,  59]], dtype=int64)

In [73]:
tree_report = metrics.classification_report(y_test, y_pred)
print(tree_report)

              precision    recall  f1-score   support

           2       0.97      0.97      0.97       109
           4       0.95      0.95      0.95        62

    accuracy                           0.96       171
   macro avg       0.96      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171

